In [12]:
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

25/02/25 06:10:08 WARN Utils: Your hostname, spark2 resolves to a loopback address: 127.0.1.1; using 192.168.69.5 instead (on interface enp0s1)
25/02/25 06:10:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/02/25 06:10:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
!head -n 1001 fhvhv_tripdata_2021-01.csv > head.csv

In [4]:
import pandas as pd
pd.DataFrame.iteritems = pd.DataFrame.items

In [5]:
df_pandas = pd.read_csv('head.csv')

In [6]:
df_pandas.dtypes

hvfhs_license_num        object
dispatching_base_num     object
pickup_datetime          object
dropoff_datetime         object
PULocationID              int64
DOLocationID              int64
SR_Flag                 float64
dtype: object

In [7]:
spark.createDataFrame(df_pandas).schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropoff_datetime', StringType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('SR_Flag', DoubleType(), True)])

In [34]:
from pyspark.sql.types import StructType, StructType, StructField, StringType, IntegerType, TimestampType, BooleanType

In [9]:
schema = StructType([
    StructField('hvfhs_license_num', StringType(), True),
    StructField('dispatching_base_num', StringType(), True),
    StructField('pickup_datetime', TimestampType(), True),
    StructField('dropoff_datetime', TimestampType(), True),
    StructField('PULocationID', IntegerType(), True),
    StructField('DOLocationID', IntegerType(), True),
    StructField('SR_Flag', StringType(), True)
])

In [10]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('head.csv')

In [11]:
df.head(5)

[Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 33, 44), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 49, 7), PULocationID=230, DOLocationID=166, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 55, 19), dropoff_datetime=datetime.datetime(2021, 1, 1, 1, 18, 21), PULocationID=152, DOLocationID=167, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 23, 56), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 38, 5), PULocationID=233, DOLocationID=142, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 42, 51), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 45, 50), PULocationID=142, DOLocationID=143, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_dat

In [13]:
df = df.repartition(24)

In [14]:
df.write.parquet("fhvhv/2021/01")

In [ ]:
df = spark.read.parquet("fhvhv/2021/01")
df


DataFrame[hvfhs_license_num: string, dispatching_base_num: string, pickup_datetime: timestamp, dropoff_datetime: timestamp, PULocationID: int, DOLocationID: int, SR_Flag: string]

In [36]:
def isodd(PULocationID):
    return PULocationID % 2 != 0

udf = f.udf(isodd, returnType=BooleanType())

In [38]:
from pyspark.sql import functions as f
df \
    .withColumn('pickup_date', f.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', f.to_date(df.dropoff_datetime)) \
    .withColumn('PULocationID_odd', udf(df.PULocationID)) \
    .select('pickup_date', 'dropoff_date', 'PULocationID', 'PULocationID_odd', 'DOLocationID') \
    .show()

+-----------+------------+------------+----------------+------------+
|pickup_date|dropoff_date|PULocationID|PULocationID_odd|DOLocationID|
+-----------+------------+------------+----------------+------------+
| 2021-01-01|  2021-01-01|         137|            true|          33|
| 2021-01-01|  2021-01-01|           4|           false|         145|
| 2021-01-01|  2021-01-01|         116|           false|          18|
| 2021-01-01|  2021-01-01|         168|           false|         265|
| 2021-01-01|  2021-01-01|          60|           false|         126|
| 2021-01-01|  2021-01-01|         263|            true|         144|
| 2021-01-01|  2021-01-01|         163|            true|         164|
| 2021-01-01|  2021-01-01|         262|           false|         263|
| 2021-01-01|  2021-01-01|         230|           false|          48|
| 2021-01-01|  2021-01-01|         254|           false|         182|
| 2021-01-01|  2021-01-01|          61|            true|         177|
| 2021-01-01|  2021-

In [21]:
df.select('pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID') \
  .where(df.hvfhs_license_num == 'HV0003') \
  .show()

+-------------------+-------------------+------------+------------+
|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|
+-------------------+-------------------+------------+------------+
|2021-01-01 00:58:37|2021-01-01 01:12:59|         137|          33|
|2021-01-01 00:00:36|2021-01-01 00:38:56|         168|         265|
|2021-01-01 00:55:13|2021-01-01 01:04:15|          60|         126|
|2021-01-01 00:31:42|2021-01-01 00:48:27|         263|         144|
|2021-01-01 00:23:28|2021-01-01 00:30:26|         230|          48|
|2021-01-01 00:53:51|2021-01-01 01:10:58|         254|         182|
|2021-01-01 00:55:03|2021-01-01 01:12:41|          61|         177|
|2021-01-01 00:23:59|2021-01-01 00:24:09|          19|          19|
|2021-01-01 00:37:19|2021-01-01 00:50:12|          22|         227|
|2021-01-01 00:10:22|2021-01-01 00:17:05|          79|         249|
|2021-01-01 00:46:05|2021-01-01 00:56:03|         230|         237|
|2021-01-01 00:41:54|2021-01-01 00:51:39|       